In [10]:
import pandas as pd
import os
import shutil
import random
import subprocess
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
random.seed(42)
def p(local_path):
    return os.path.join("/home/jupyter/mt-dnn",local_path)

In [2]:
src = pd.read_csv("~/mt-dnn/toxic_data/train.csv")

In [3]:
train, test = train_test_split(src,test_size=0.05)

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
def full_tokenizer(raw_string):
    raw_string=raw_string[0]
    tokens = tokenizer.tokenize(raw_string)
    if len(tokens)>510:
        tokens = tokens[0:510]
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    return tokenizer.convert_tokens_to_ids(tokens)

In [9]:
def toxic_prep(sample,path_to_prepared=''):
    how_many_datapoints = sample.shape[0]
    sample = sample[['id',"target",'comment_text']]        
    sample['token_id'] = sample[['comment_text']].apply(full_tokenizer,axis = 1)
    sample['type_id'] = sample[['token_id']].apply(lambda x:[0]*len(x[0]),axis = 1)
    sample.drop(['comment_text'], axis=1, inplace=True)
    sample = sample.rename(columns={'id': 'uid', 'target': 'label'})
    sample['label'] = sample[['label']].apply(lambda x:int(x>=0.5),axis = 1)
    sample.to_json(path_to_prepared, orient='records', lines=True)
toxic_prep(test,"/home/jupyter/mt-dnn/data/mt_dnn/toxic_test.json")
toxic_prep(train,"/home/jupyter/mt-dnn/data/mt_dnn/toxic_train.json")

In [5]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [5]:
file_len("/home/jupyter/mt-dnn/data/mt_dnn/toxic_test.json")

90244

In [7]:
file_len("/home/jupyter/mt-dnn/data/mt_dnn/toxic_train.json")

1714630

In [6]:
file_len("/home/jupyter/mt-dnn/data/mt_dnn/sst_test.json")

1821

In [7]:
file_len("/home/jupyter/mt-dnn/data/mt_dnn/sst_train.json")

67349

In [9]:
def cut_file(file,max_length):
    with open(file) as f, open("/home/jupyter/mt-dnn/tmp.txt", "w") as out:
        out.writelines(f.readlines()[:max_length])
    os.remove(file)
    shutil.move("/home/jupyter/mt-dnn/tmp.txt",file)    

In [10]:
# trying a shorter train set, for more reasonable training time
cut_file('/home/jupyter/mt-dnn/data/mt_dnn/toxic_test.json',2000)

In [ ]:
#Not working... Cuda ran out of memory after 3 hours...
#I'll try instead to divide the train set into much smaller batches, to have shorter epochs

In [31]:
for i in range(3):
    print(i)

0
1
2


In [42]:
def smaller_train_set(file_in,file_out,batch_size=4000,batch_number=0):
    with open(file_in) as f:
        lines=f.readlines()
        with open(file_out, "w") as out:
            out.writelines(lines[batch_size*batch_number:batch_size*(batch_number+1)])

In [43]:
smaller_train_set(p("data/mt_dnn/toxic_test.json"),p("data/mt_dnn/toxic_test0.json"))
smaller_train_set(p("data/mt_dnn/toxic_train.json"),p("data/mt_dnn/toxic_train0.json"),batch_size=140000)

In [44]:
file_len(p("data/mt_dnn/toxic_test0.json"))

4000

In [45]:
file_len(p("data/mt_dnn/toxic_train0.json"))

140000

In [19]:
# sh scripts/run_toxic_mini_trains.sh 8 0

In [30]:
[0]*6

[0, 0, 0, 0, 0, 0]

In [6]:
!sh scripts/run_toxic_mini.sh 4 0

scripts/run_toxic_mini.sh: 2: scripts/run_toxic_mini.sh: [[: not found
!!!!BERT_PATH:mt_dnn_models/mt_dnn_large.pt
export CUDA_VISIBLE_DEVICES=0
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Namespace(answer_att_hidden_size=128, answer_att_type='bilinear', answer_dropout_p=0.1, answer_mem_drop_p=0.1, answer_mem_type=1, answer_merge_opt=1, answer_num_turn=5, answer_opt=0, answer_rnn_type='gru', answer_sum_att_type='bilinear', answer_weight_norm_on=False, batch_size=4, batch_size_eval=4, bert_dropout_p=0.1, bert_l2norm=0.0, cuda=True, data_dir='data/mt_dnn', data_sort_on=False, dropout_p=0.1, dropout_w=0.0, dump_state_on=False, ema_gamma=0.995, ema_opt=0, embedding_opt=0, epochs=1, freeze_layers=-1, global_grad_clipping=1.0, grad_clipping=0.0, have_lr_scheduler=True, init_checkpoint='mt_dnn_models/mt_dnn_large.pt', init_ratio=1, label_size='3', learning_rate=5e-05, log_file='checkpoints/mt-dnn-toxic_adamax_answer_opt0_gc0_ggc1_2019-06-02T0741/l